# HMS - harmful brain activity

The aim of this project is to create and train a deep learning model that will detect harmful brain activity from EEG signal

Libraries we're going to use

`pandas` - a library for reading and processing data frames and input data files
`numpy` - mathematical library for efficient multidimensional algebra and raw data processing
`pytorch` - deep learning library and framework that allows for efficient data processing and neural network training using cpu or gpu
`scikit-learn` - sklearn - Rich in functionality machine learning and data science library. Industry standard for ML
`scipy` - scientific library including many mathematical tools. For example for signal processing
`os` - operating system packet that allows for navigating in the file system from the level of python

In [1]:
import pandas as pd
import numpy as np
import torch
import sklearn
import scipy
import os

We want to fully utilise our resources. We use `cuda` - a tool used by `pytorch` to perform computation on gpu - this way neural networks can be trained much faster

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

# Loading the data

The data comes from [kaggle](https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification) competition. 

 It consists of `.parquet` files with eeg signal data as well as plotted spectrograms of the signals.

In [3]:
SAMPLING_FREQUENCY = 200
SAMPLES_IN_MEASUREMENT = 10000

In [4]:
# directory that contains data from kaggle hms
INPUT_DATA_DIR = "data"

# directory in which our npy files are/will be stored - this will allow for faster loading of the data later
PROCESSED_DATA_DIR = "processed_data"

The metadata is stored in train.csv and test.csv files

In [5]:
train_meta_full = pd.read_csv(INPUT_DATA_DIR + "/train.csv")
train_meta = train_meta_full.loc[train_meta_full["eeg_sub_id"] == 0]

test_meta = pd.read_csv(INPUT_DATA_DIR + "/test.csv")

Signal data tends to be very noisy. That's why it's important to filter the noise before processing the data further. For this we'll use the butterworth lowpass filter.

Lowpass filter filters out high and noisy frequencies. It only allows the frequencies lower than specified to pass through it.

The data is sampled at the frequency of 200Hz, we want to cut off noise frequencies higher than 20Hz

In [6]:
cutoff_freq = 20
sampling_rate = 200
order = 4
lowcut = 0.5
highcut = 20

b, a = scipy.signal.butter(
    order, (lowcut, highcut), btype="bandpass", analog=False, fs=sampling_rate
)

Extract `.parquet` data.

The training data is stored in `.parquet` files as individual read values of separate electrodes placed on patient's head during EEG examination.

Normally specialists don't analyze the separate signals but rather differences between the neighbouring electrodes. 

`extract_parquet` function computes those differences and creates time series of those differences as opposed to pure signal.

Those differences are then processed with previously initialized lowpass filter, clipped so that their values are not too high and converted to pytorch tensor objects.

We want to use pytorch tensors because it's the object class required to perform neural network training.

In [7]:
# take a parquet dataframe and compute correct values for each column
# we want columns such as "Fp1-F7" as can be seen in /example_figures
def extract_parquet(parquet_data: torch.tensor):
    parquet_data["Fp1-F7"] = parquet_data["Fp1"] - parquet_data["F7"]
    parquet_data["F7-T3"] = parquet_data["F7"] - parquet_data["T3"]
    parquet_data["T3-T5"] = parquet_data["T3"] - parquet_data["T5"]
    parquet_data["T5-O1"] = parquet_data["T5"] - parquet_data["O1"]

    parquet_data["Fp2-F8"] = parquet_data["Fp2"] - parquet_data["F8"]
    parquet_data["F8-T4"] = parquet_data["F8"] - parquet_data["T4"]
    parquet_data["T4-T6"] = parquet_data["T4"] - parquet_data["T6"]
    parquet_data["T6-O2"] = parquet_data["T6"] - parquet_data["O2"]

    parquet_data["Fp1-F3"] = parquet_data["Fp1"] - parquet_data["F3"]
    parquet_data["F3-C3"] = parquet_data["F3"] - parquet_data["C3"]
    parquet_data["C3-P3"] = parquet_data["C3"] - parquet_data["P3"]
    parquet_data["P3-O1"] = parquet_data["P3"] - parquet_data["O1"]

    parquet_data["Fp2-F4"] = parquet_data["Fp2"] - parquet_data["F4"]
    parquet_data["F4-C4"] = parquet_data["F4"] - parquet_data["C4"]
    parquet_data["C4-P4"] = parquet_data["C4"] - parquet_data["P4"]
    parquet_data["P4-O2"] = parquet_data["P4"] - parquet_data["O2"]

    parquet_data["Fz-Cz"] = parquet_data["Fz"] - parquet_data["Cz"]
    parquet_data["Cz-Pz"] = parquet_data["Cz"] - parquet_data["Pz"]

    parquet_data = parquet_data.drop(
        [
            "Fp1",
            "F3",
            "C3",
            "P3",
            "F7",
            "T3",
            "T5",
            "O1",
            "Fz",
            "Cz",
            "Pz",
            "Fp2",
            "F4",
            "C4",
            "P4",
            "F8",
            "T4",
            "T6",
            "O2",
        ],
        axis=1,
    )

    # we want to reorder the columns so that the EKG signal is at the end.
    idx = parquet_data.columns[1:].to_list() + [parquet_data.columns[0]]

    # we want to transpose the values so that they're easier to handle later
    parquet_data = parquet_data[idx].values.T

    # filter the high frequencies
    parquet_data = scipy.signal.lfilter(b, a, parquet_data, axis=0)

    # convert to pytorch tensor
    parquet_data = torch.from_numpy(parquet_data).type(torch.float32)

    # clip high values to 1024
    parquet_data = torch.clip(parquet_data, -1024, 1024)

    return parquet_data

We need to extract the data from the `.parquet` files, so in a loop we'll iterate through the metadata and extract the data and the training labels.

In [8]:
# # A list for data entries from different files
# eeg_data = []
# 
# # indices of the dataframes with too many missing values
# faulty_eeg_id = []
# 
# if not os.path.exists(f"{PROCESSED_DATA_DIR}/eeg_labels.pt"):
#     os.makedirs(PROCESSED_DATA_DIR,exist_ok=True)
# 
#     # iterate through IDs of eeg and extract each .parquet file
#     for eeg_id in train_meta["eeg_id"]:
#         # open the file using pandas
#         parquet_data = pd.read_parquet(INPUT_DATA_DIR + f"/train_eegs/{eeg_id}.parquet")
# 
#         # fill the missing values and extract the first 10000 measurements
#         parquet_data = parquet_data.interpolate(method="ffill")[:10000]
# 
#         # If at this point there are any missing values, the file can't be used
#         if np.any(parquet_data.isna()):
#             faulty_eeg_id.append(eeg_id)
#             continue
# 
#         # we call the preprocessing function
#         eeg = extract_parquet(parquet_data)
# 
#         # and add the data to the list of processed entries
#         eeg_data.append(eeg)
# 
#     # convert the list of tensors to one tensor
#     eeg_data = torch.stack(eeg_data)
# 
#     # save to PROCESSED_DATA_TIR
#     torch.save(eeg_data, f"{PROCESSED_DATA_DIR}/eeg_data.pt")
# 
#     # extract labels for valid entries
#     all_labels = train_meta.loc[~train_meta["eeg_id"].isin(faulty_eeg_id)]
#     eeg_labels = all_labels[
#         ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
#     ].values
# 
#     # convert labels to pytorch tensor
#     eeg_labels = torch.tensor(np.array(eeg_labels), dtype=torch.float32)
# 
#     # The labels are probability - they must sum to 1
#     eeg_labels = eeg_labels / eeg_labels.sum(dim=1, keepdims=True)
# 
#     # save for easier training later
#     torch.save(eeg_labels, f"{PROCESSED_DATA_DIR}/eeg_labels.pt")
# else:
#     eeg_data = torch.load(f"{PROCESSED_DATA_DIR}/eeg_data.pt")
#     eeg_labels = torch.load(f"{PROCESSED_DATA_DIR}/eeg_labels.pt")

In [9]:
def retrieve_eeg_dataset():
    if not os.path.exists(f"{PROCESSED_DATA_DIR}/eeg_labels_full.pt"):
        os.makedirs(PROCESSED_DATA_DIR,exist_ok=True)
        
        eeg_data_full = []
        eeg_labels_full = []
        eeg_id_measurements = []
        
        i = 0
        # iterate through IDs of eeg and extract each .parquet file
        for eeg_id in train_meta_full["eeg_id"].unique():
            if i % 500 == 0: print(i)
            i+=1
            # open the file using pandas
            parquet_data = pd.read_parquet(INPUT_DATA_DIR + f"/train_eegs/{eeg_id}.parquet")
            
            # read and interpolate the data
            parquet_data = extract_parquet(parquet_data.interpolate(method="ffill"))
            
            for idx, row in train_meta_full.loc[train_meta_full["eeg_id"]==eeg_id].iterrows():
                offset = int(200*row["eeg_label_offset_seconds"])
                
                parquet_window = parquet_data[:,offset:offset+10000]
    
                # If at this point there are any missing values, the file can't be used
                if torch.any(parquet_window.isnan()):
                    continue
        
                # and add the data to the list of processed entries
                eeg_data_full.append(parquet_window)
                eeg_labels_full.append(row[["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]].values)
                eeg_id_measurements.append(eeg_id)
        
        
        # convert the list of tensors to one tensor
        print("stacking eeg")
        eeg_data_full = torch.stack(eeg_data_full)
    
        # save to PROCESSED_DATA_TIR
        print("saving eeg")
        torch.save(eeg_data_full, f"{PROCESSED_DATA_DIR}/eeg_data_full.pt")
        
        # stack labels to one numpy array and convert to pytorch tensor
        print("stacking labels")
        eeg_labels_full = np.vstack(eeg_labels_full)
        print("converting labels")
        eeg_labels_full = torch.from_numpy(eeg_labels_full.astype(np.float32))
    
        # The labels represent probability - they must sum to 1
        print("labels -> distribution")
        eeg_labels_full = eeg_labels_full / eeg_labels_full.sum(dim=1, keepdims=True)
    
        # save for easier reuse later
        print("saving labels")
        torch.save(eeg_labels_full, f"{PROCESSED_DATA_DIR}/eeg_labels_full.pt")
        
        eeg_id_measurements = torch.tensor(np.array(eeg_id_measurements))
        torch.save(eeg_id_measurements,f"{PROCESSED_DATA_DIR}/eeg_id_measuerements.pt")
    else:
        eeg_data_full = torch.load(f"{PROCESSED_DATA_DIR}/eeg_data_full.pt")
        eeg_labels_full = torch.load(f"{PROCESSED_DATA_DIR}/eeg_labels_full.pt")
        eeg_id_measurements = torch.load(f"{PROCESSED_DATA_DIR}/eeg_id_measuerements.pt")
    return eeg_data_full, eeg_labels_full, eeg_id_measurements

In [10]:
from sklearn.model_selection import train_test_split

if not os.path.exists(f"{PROCESSED_DATA_DIR}/labels_valid.pt"):
    eeg_data_full,eeg_labels_full, eeg_id_measurements = retrieve_eeg_dataset()
    
    id_train, id_test = train_test_split(
        torch.unique(eeg_id_measurements), test_size=0.2
    )
    
    id_test, id_valid = train_test_split(
        id_test, test_size=0.5
    )
    
    train_idx = torch.isin(eeg_id_measurements, id_train)
    test_idx = torch.isin(eeg_id_measurements, id_test)
    valid_idx = torch.isin(eeg_id_measurements, id_valid)
    del eeg_id_measurements
    
    X_train, y_train = eeg_data_full[train_idx], eeg_labels_full[train_idx]
    X_test, y_test = eeg_data_full[test_idx], eeg_labels_full[test_idx]
    X_valid, y_valid = eeg_data_full[valid_idx], eeg_labels_full[valid_idx]
    del eeg_data_full
    del eeg_labels_full
    
    torch.save(X_train,f"{PROCESSED_DATA_DIR}/eeg_train.pt")
    torch.save(y_train,f"{PROCESSED_DATA_DIR}/labels_train.pt")
    torch.save(X_test,f"{PROCESSED_DATA_DIR}/eeg_test.pt")
    torch.save(y_test,f"{PROCESSED_DATA_DIR}/labels_test.pt")
    torch.save(X_valid,f"{PROCESSED_DATA_DIR}/eeg_valid.pt")
    torch.save(y_valid,f"{PROCESSED_DATA_DIR}/labels_valid.pt")
else:
    X_train = torch.load(f"{PROCESSED_DATA_DIR}/eeg_train.pt")
    y_train = torch.load(f"{PROCESSED_DATA_DIR}/labels_train.pt")
    X_test = torch.load(f"{PROCESSED_DATA_DIR}/eeg_test.pt")
    y_test = torch.load(f"{PROCESSED_DATA_DIR}/labels_test.pt")
    X_valid = torch.load(f"{PROCESSED_DATA_DIR}/eeg_valid.pt")
    y_valid = torch.load(f"{PROCESSED_DATA_DIR}/labels_valid.pt")

---

# Creating a data loader, preprocessing

Using scikit-learn we'll split the data into training and testing dataset

In [11]:
# from sklearn.model_selection import train_test_split
# 
# X_train, X_test, y_train, y_test = train_test_split(
#     eeg_data_full, eeg_labels_full, test_size=0.2
# )
# 
# X_test, X_valid, y_test, y_valid = train_test_split(
#     X_test, y_test, test_size=0.5
# )


To perform the computation on gpu, we have to move the data into cuda

In [12]:
# X_train, y_train = X_train.to(DEVICE), y_train.to(DEVICE)
# X_valid, y_valid = X_valid.to(DEVICE), y_valid.to(DEVICE)
# X_test, y_test = X_test.to(DEVICE), y_test.to(DEVICE)

Create a HMS dataset class that will help us load the data during the model training

In [13]:
from torch.utils.data import DataLoader, Dataset

BATCH_SIZE = 512

class CustomImageDataset(Dataset):
    def __init__(self, X, y, train = False):
        self.X = X
        self.y = y
        self.train = train

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx] 

train_loader = DataLoader(
    CustomImageDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True
)

test_loader = DataLoader(
    CustomImageDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=True
)

valid_loader = DataLoader(
    CustomImageDataset(X_valid, y_valid), batch_size=BATCH_SIZE, shuffle=True
)

---

# Creating a model

We create a machine learning model - a python object that will be trained to predict correct data labels

In [14]:
import torch.nn as nn

class ConvModel(nn.Module):
    def __init__(self):
        super(ConvModel, self).__init__()

        self.conv1 = nn.Sequential(
            # (19, 10000)
            nn.Conv1d(19, 76, kernel_size=5,padding=2,groups=19),
            nn.MaxPool1d(2,ceil_mode=True),
            nn.BatchNorm1d(76),
        )
        
        self.conv2 = nn.Sequential(
            # (64, 5000)
            nn.Conv1d(76, 64, kernel_size=5,padding=2),
            nn.MaxPool1d(2,ceil_mode=True),
            nn.BatchNorm1d(64),
        )
        
        self.conv3 = nn.Sequential(
            # (64, 5000)
            nn.Conv1d(64, 64, kernel_size=5,padding=2),
            nn.MaxPool1d(2,ceil_mode=True),
            nn.BatchNorm1d(64),
        )
        
        self.conv4 = nn.Sequential(
            # (64, 1250)
            nn.Conv1d(64, 32, kernel_size=3,padding=1),
            nn.MaxPool1d(2,ceil_mode=True),
            nn.BatchNorm1d(32),
            # (64,625)
        )

        
        self.mlp = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            
            nn.Linear(20000,4096),
            nn.Dropout(0.5),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096,4096),
            nn.Dropout(0.5),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, 6),
            
        )      

    # input in CHW / CW format
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.mlp(x)
        return x

---

# Training

After data preprocessing and buikding the model we need to train it. 

We'll use a [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) as a loss function. 
KLDivergence is the widely used metric suggested by the scientists who worked on the best solutions for the HMS competition.

We also use Adam optimizier - an adaptive learning rate optimization algorithm used in training deep learning models.

It combines the advantages of two other extensions of stochastic gradient descent 
- (SGD): Adaptive Gradient Algorithm (AdaGrad) 
- Root Mean Square Propagation (RMSProp)

It adjusts the learning rate for each parameter dynamically, making it efficient and well-suited for large datasets and complex models.

In [15]:
from torch.optim import Adam

# initialize the model and move it to gpu
model = ConvModel().to(DEVICE)

In [16]:
softmax = nn.Softmax(1)
log_softmax = lambda x: nn.functional.log_softmax(x,dim=1)

In [17]:
def rate_model(loader, loss_function, activation):
    model.eval()
    with torch.no_grad():
        iteration = 0
        total_loss = 0
        for batch in loader:
            # extract data and labels
            batch_data, batch_labels = batch
            batch_data = batch_data.to(DEVICE)
            batch_labels = batch_labels.to(DEVICE)
    
            # perform the inference on the data
            prediction = model(batch_data)
            prediction = activation(prediction)
    
            # compare the inference output with real labels and calculate the value of loss function
            loss = loss_function(prediction, batch_labels)
    
            # accumulate loss
            total_loss += float(loss.item())
    
            iteration += 1
    model.train()
    return total_loss / iteration

## Pretraining 1 - regression

In [18]:
# Create an optimizer that will perform the gradient descent algorithm
optimizer = Adam(model.parameters(), lr=1e-4)

loss_function = nn.MSELoss()

In [ ]:
EPOCHS = 100

model.train()

# we train the network for EPOCHS epochs
for epoch in range(EPOCHS):
    # variables that will let us compute average epoch loss
    iteration = 0
    total_loss = 0
    
    train_losses_pretraining = []
    valid_losses_pretraining = []
    
    y_noise = torch.normal(0,0.05,size=(1,6)).to(DEVICE)
    
    # iterate through data batches in the data loader
    for batch in train_loader:
        # extract data and labels
        batch_data, batch_labels = batch
        batch_data = batch_data.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)

        # perform the inference on the data
        prediction = model(batch_data)
        prediction = softmax(prediction)
        
        # compare the inference output with real labels and calculate the value of loss function
        loss = loss_function(prediction, batch_labels + y_noise)

        # accumulate loss to print the average at the end of the epoch
        total_loss += float(loss.item())

        # reset the optimizer
        optimizer.zero_grad()

        # compute the error backpropagation
        loss.backward()

        # update the weights of the model
        optimizer.step()

        iteration += 1
        
    train_losses_pretraining.append(total_loss / iteration)
    valid_losses_pretraining.append(rate_model(valid_loader,loss_function,softmax))

    print(f"epoch : {epoch}")
    print(f"train loss : {train_losses_pretraining[-1]}")
    print(f"valid loss : {valid_losses_pretraining[-1]}")
    print("\n")

epoch : 0
train loss : 0.08995196968317032
valid loss : 0.08615538477897644

epoch : 1
train loss : 0.08300052081996745
valid loss : 0.08118167626006263

epoch : 2
train loss : 0.08007561614116034
valid loss : 0.08087311010985147

epoch : 3
train loss : 0.07578116086396304
valid loss : 0.07752276176498049

epoch : 4
train loss : 0.07251161662015047
valid loss : 0.0777854160183952

epoch : 5
train loss : 0.07010479068214243
valid loss : 0.07524949596041725

epoch : 6
train loss : 0.06740692463336569
valid loss : 0.0766282014193989

epoch : 7
train loss : 0.06474960358305411
valid loss : 0.07696112671068736

epoch : 8
train loss : 0.06206686611879956
valid loss : 0.078645924727122
epoch : 9
train loss : 0.05950098629250671
valid loss : 0.07485842314504441

epoch : 10
train loss : 0.05641151601166436
valid loss : 0.07537735998630524

epoch : 11
train loss : 0.05354363356124271
valid loss : 0.07429902007182439



In [ ]:
from matplotlib import pyplot as plt

os.makedirs(f"{PROCESSED_DATA_DIR}/results",exist_ok=True)

torch.save(torch.tensor(train_losses_pretraining),f"{PROCESSED_DATA_DIR}/results/train_losses_pretraining.pt")
torch.save(torch.tensor(valid_losses_pretraining),f"{PROCESSED_DATA_DIR}/results/valid_losses_pretraining.pt")

plt.plot(train_losses_pretraining)
plt.plot(valid_losses_pretraining)

plt.tight_layout()
plt.savefig(f"{PROCESSED_DATA_DIR}/results/pretraining.pdf")

In [ ]:
rate_model(test_loader,loss_function,softmax)

---

## Final training

In [ ]:
# Create an optimizer that will perform the gradient descent algorithm
optimizer = Adam(model.parameters(), lr=1e-5)

# As a loss function we'll be using KLDivergence
# loss_function = nn.KLDivLoss(reduction="batchmean", log_target=False)
loss_function = nn.KLDivLoss(reduction="batchmean",log_target=False)

In [ ]:
EPOCHS = 200

model.train()

# we train the network for EPOCHS epochs
for epoch in range(EPOCHS):
    # variables that will let us compute average epoch loss
    iteration = 0
    total_loss = 0

    train_losses_training = []
    valid_losses_training = []

    # iterate through data batches in the data loader
    for batch in train_loader:
        # extract data and labels
        batch_data, batch_labels = batch

        # perform the inference on the data
        prediction = model(batch_data)
        batch_data = batch_data.to(DEVICE)
        batch_labels = batch_labels.to(DEVICE)

        # compare the inference output with real labels and calculate the value of loss function
        loss = loss_function(nn.functional.log_softmax(prediction, dim=1), batch_labels)

        # accumulate loss to print the average at the end of the epoch
        total_loss += float(loss.item())

        # reset the optimizer
        optimizer.zero_grad()

        # compute the error backpropagation
        loss.backward()

        # update the weights of the model
        optimizer.step()

        iteration += 1

    train_losses_training.append(total_loss / iteration)
    valid_losses_training.append(rate_model(valid_loader,loss_function,log_softmax))

    print(f"epoch : {epoch}")
    print(f"train loss : {train_losses_training[-1]}")
    print(f"valid loss : {valid_losses_training[-1]}")
    print("\n")
    

In [ ]:
torch.save(torch.tensor(train_losses_training),f"{PROCESSED_DATA_DIR}/results/train_losses_training.pt")
torch.save(torch.tensor(valid_losses_training),f"{PROCESSED_DATA_DIR}/results/valid_losses_training.pt")

os.makedirs(f"{PROCESSED_DATA_DIR}/results")

plt.plot(train_losses_training)
plt.plot(valid_losses_training)

plt.tight_layout()
plt.savefig(f"{PROCESSED_DATA_DIR}/results/training.pdf")

In [ ]:
rate_model(test_loader,loss_function,log_softmax)